In [9]:
import numpy as np
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline
import spacy

In [10]:
from JOSS_PDF_Cleaner import Clean_PDF
import re
from termcolor import colored
import warnings

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import string
import glob
from tqdm import tqdm 
#import pdfminer
from pdfminer.high_level import extract_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
#nltk.download('stopwords')
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
#nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
from nltk.probability import FreqDist

In [12]:
#from Master_Methods import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import string
import glob
from tqdm import tqdm 
#import pdfminer
from pdfminer.high_level import extract_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
nltk.download('stopwords')
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.probability import FreqDist
from JOSS_PDF_Cleaner import Clean_PDF
import re
from termcolor import colored
import spacy



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

# Import Files

In [14]:
# https://zenodo.org/record/3631674#.YeglfP7P2Uk
df = pd.read_csv('/Volumes/Seagate Backup Plus Drive/JOSS Project/wiki_tfidf_terms.csv')
df_reviewers = pd.read_csv('../JOSS_Reviewer_Matcher/Data/JOSS Table Test.csv')

# Select Paper to find Reviewers for

In [127]:
#PAPER_OF_INTEREST_FNAME  = glob.glob('/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/*/*/*.pdf')
#K = 260
#Paper_interest = PAPER_OF_INTEREST_FNAME[K] # Replace with path to paper of interest
Paper_interest = '/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/joss-papers-master/joss.01539/10.21105.joss.01539.pdf'
print(Paper_interest)

/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/joss-papers-master/joss.01539/10.21105.joss.01539.pdf


# Printing Paper of Interest

In [128]:
def MakeGreenRedText(Paper_interest, printer=True):
    texts = ''
    arr = []
    from pdfminer.high_level import extract_pages
    from pdfminer.layout import LTTextContainer
    for page_layout in extract_pages(Paper_interest):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                score = Clean_PDF(element.get_text().lower())
            #print(score)
                if score == 0:
                    if printer == True:
                        print(colored(element.get_text().lower(), 'green'))
                    arr.append(element.get_text())
                    texts = texts  + element.get_text() + ' '
                else:
                    if printer == True:
                        print(colored(element.get_text().lower(), 'red'))
            #arr.append(element.get_text())
            #texts = texts  + element.get_text() + ' '
    arr = np.array(arr)
    return texts, arr

In [129]:
texts, arr = MakeGreenRedText(Paper_interest)

doi: 10.21105/joss.01539

software

• review
• repository
• archive

submitted: 25 may 2019
published: 24 september 2019

license
authors of papers retain
copyright and release the work
under a creative commons
attribution 4.0 international
license (cc-by).

pulse: a python package based on fenics for solving
problems in cardiac mechanics

henrik nicolay topnes finsberg1

1 simula research laboratory, oslo, norway

summary

pulse is a software package based on fenics (logg, mardal, & wells, 2012) that aims to
solve problems in cardiac mechanics (but is easily extended to solve more general problems in
continuum mechanics). pulse is a result of the author’s phd thesis (h. n. finsberg, 2017),
where most of the relevant background for the code can be found.

while fenics offers a general framework for solving pdes, pulse specifically targets problems
in continuum mechanics. therefore, most of the code for applying compatible boundary
conditions, formulating the governing equations, choosi

# All Functions
Hidden for clarity

In [130]:
################# Idea 1 #####################
def Get_Lemma_Words(POI_PDF):
  text = str(POI_PDF)
  text2 = text.split()
  words_no_punc = []

  for w in text2:
    if w.isalpha():
      words_no_punc.append(w.lower())
  from nltk.corpus import stopwords
  stopwords = stopwords.words('english')  
  clean_words = []
  for w in words_no_punc:
    if w not in stopwords:
      clean_words.append(w)
  clean_words_arr = ''
  for i in range(len(clean_words)):
    clean_words_arr = clean_words_arr + ' ' + str(clean_words[i])

  string_for_lemmatizing = clean_words_arr
  lemmatizer = WordNetLemmatizer()
  words_2 = word_tokenize(string_for_lemmatizing)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words_2]

  lemmatized_words_arr = ''
  for i in range(len(lemmatized_words)):
    lemmatized_words_arr = lemmatized_words_arr + ' ' + str(lemmatized_words[i])
  words = word_tokenize(lemmatized_words_arr)

  return words

def Get_reviewer_sample_tf(Paper_interest, df_reviewers=df_reviewers,num_suggestions=5, num_top20=20):
    POI_PDF = [extract_text(Paper_interest)]
    text = str(POI_PDF)
    words =  Get_Lemma_Words(POI_PDF)
    #print(len(words))
    fdist = FreqDist(words)
    X = np.array(fdist.most_common())
    top20_tf = X[:num_top20,0]
    match_arr = Compare_topics(top20_tf, df_reviewers)
    top5_reviewers = np.argsort(match_arr)[-num_suggestions:]

    
    all_usernames = []
    all_domains = []
    all_num_matched_words = []
    all_matched_words = []
    for i in range(num_suggestions):
      K = -1*(i+1)
      index = top5_reviewers[K]
      #print(i)
      t =df_reviewers.iloc[index+1]['Domains/topic areas you are comfortable reviewing'].lower()
      
      all_usernames.append(df_reviewers.username.iloc[index+1])
      all_domains.append(t)
      all_num_matched_words.append(match_arr[index])

      uniarr = Split_columns(t)
      matched_words = []
      #print(uniarr)
      for j in range(len(uniarr)):
        for k in range(len(top20_tf)):
          if uniarr[j] == top20_tf[k]:
            matched_words.append(uniarr[j])
      all_matched_words.append(matched_words)

    #df_reviewers.username.iloc[+1]

    return all_usernames, all_domains, all_num_matched_words, all_matched_words

def Compare_topics(top20, df_reviewers):
  length = df_reviewers.shape[0] - 1
  match_arr = np.zeros(length)
  for i in range(length):
    if pd.isna(df_reviewers['Domains/topic areas you are comfortable reviewing'].str.lower().values[1+i]) == False:
      t = df_reviewers['Domains/topic areas you are comfortable reviewing'].str.lower().values[1+i]
      #print(i)
      uniarr = Split_columns(t)
      for j in range(len(uniarr)):
        for k in range(len(top20)):
          if uniarr[j] == top20[k]:
            match_arr[i] = match_arr[i] + 1
  return match_arr

def Split_columns(t):
  txt = " ".join("".join([" " if ch in string.punctuation else ch for ch in t]).split())
  sol1 = np.char.split(txt, ' ')
  txt_arr  = array_of_lists_to_array(sol1)
  uniarr = np.unique(txt_arr)
  return uniarr

def array_of_lists_to_array(arr):
    return np.apply_along_axis(lambda a: np.array(a[0]), -1, arr[..., None])


def summatation_bot_tf(all_usernames, all_domains, all_num_matched_words, all_matched_words):
  length = len(all_usernames)
  message = 'Hello. \nI have found ' + str(length) + ' possible reviewers for this paper.' +'\n\n'
  for i in range(length):
    ps = 'I believe ' + colored(all_usernames[i], 'green') + ' will make a good reviewer for this paper because they have matched ' + colored(str(int(all_num_matched_words[i])), 'blue') +  ' words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ' + colored(str(all_matched_words[i]), 'blue') +'.\nFrom their topics domain: ' + colored(str(all_domains[i].replace('\n', ', ')), 'red') +'.\n'
    message = message + ps + '\n'
  print(message)

#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 2 #####################

# Idea 2

def Get_reviewer_sample_tf_idf(Paper_interest, df=df, df_reviewers=df_reviewers, num_suggestions=5, num_top20=20):
    POI_PDF = [extract_text(Paper_interest)]
    text = str(POI_PDF)
    words = Get_Lemma_Words(POI_PDF)
    #print(len(words))
    fdist = FreqDist(words)
    X = np.array(fdist.most_common())
    tf_idf_arr_names, tf_idf_arr_floats = determine_wiki_td_idf(X, df=df)
    #print('determined wiki')
    num_arr = np.array(tf_idf_arr_floats)
    tf_idf_arr_names_arr = np.array(tf_idf_arr_names)
    top20_tf_idf = tf_idf_arr_names_arr[np.argsort(num_arr)[:20]]
    match_arr = Compare_topics(top20_tf_idf, df_reviewers)
    top5_reviewers = np.argsort(match_arr)[-num_suggestions:]
    
    
    all_usernames = []
    all_domains = []
    all_num_matched_words = []
    all_matched_words = []
    for i in range(num_suggestions):
      K = -1*(i+1)
      index = top5_reviewers[K]
      #print(i)
      t =df_reviewers.iloc[index+1]['Domains/topic areas you are comfortable reviewing'].lower()
      
      all_usernames.append(df_reviewers.username.iloc[index+1])
      all_domains.append(t)
      all_num_matched_words.append(match_arr[index])

      uniarr = Split_columns(t)
      matched_words = []
      #print(uniarr)
      for j in range(len(uniarr)):
        for k in range(len(top20_tf_idf)):
          if uniarr[j] == top20_tf_idf[k]:
            matched_words.append(uniarr[j])
      all_matched_words.append(matched_words)

    #df_reviewers.username.iloc[+1]

    return all_usernames, all_domains, all_num_matched_words, all_matched_words

def determine_wiki_td_idf(x, df=df):
    tf_idf_arr_names = []
    tf_idf_arr_floats = []
    for i in range(len(x)):
        if df[df['token'] ==x[i][0]].frequency.empty == False:
            wiki_tf = df[df['token'] ==x[i][0]].frequency.values[0]
            doc_tf = int(x[i][1])
            tf_idf = np.log(wiki_tf/doc_tf)
            tf_idf_arr_names.append(x[i][0])
            tf_idf_arr_floats.append(tf_idf)
    return tf_idf_arr_names, tf_idf_arr_floats

def summatation_bot_tf_idf(all_usernames, all_domains, all_num_matched_words, all_matched_words):
  length = len(all_usernames)
  message = 'Hello. \nI have found ' + str(length) + ' possible reviewers for this paper.' +'\n\n'
  for i in range(length):
    ps = 'I believe ' + colored(all_usernames[i], 'green') + ' will make a good reviewer for this paper because they have matched ' + colored(str(int(all_num_matched_words[i])), 'blue') +  ' words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ' + colored(str(all_matched_words[i]), 'blue') +'.\nFrom their topics domain: ' + colored(str(all_domains[i].replace('\n', ', ')), 'red') +'.\n'
    message = message + ps + '\n'
  print(message)


#########################################################################################################################################################

#########################################################################################################################################################

################# Gensim ####################

def preprocess_gensim(text):
    stop_words = stopwords.words('english')
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

def W2V_Gensim_Processing(texts,model, print_outside_corpus=True):
    t = texts
    t = t.replace('-\\n','')
    t = t.replace('-\n','')
    t = t.replace('\\n',' ')
    t = t.replace('\n', ' ')
    texts = t
    #print(t)

    #download('punkt') #tokenizer, run once
    #download('stopwords') #stopwords dictionary, run once
    stop_words = stopwords.words('english')

    texts = [texts]
    corpus = [preprocess_gensim(text) for text in texts]

    sum_vector_text = np.zeros((300))
    counter = 0
    for i in range(len(corpus[0])):
        if corpus[0][i] in model.vocab:
            sum_vector_text = sum_vector_text + model.wv[corpus[0][i]]
            counter = counter + 1
        else:
            if print_outside_corpus == True:
                print(corpus[0][i])
    average_vector_text = sum_vector_text/ counter
    return average_vector_text


def GetReviewerSample_W2V_Gensim(paper_vec, df_reviewers=df_reviewers):
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    all_usernames = []
    all_domains = []
    all_cosine_sims = []
    for j in range(df_reviewers.shape[0]-1):
        if pd.isna(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing']) == False:
            reviewer_interests = df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing'].lower()
            reviewer_interests.replace('/',' ')
            reviewer_corpus = [preprocess_gensim(reviewer_interests)]
            if bool(reviewer_corpus[0]) == True:
        #print(reviewer_corpus)
                sum_vector_text = np.zeros((300))
                counter = 0
                for i in range(len(reviewer_corpus[0])):
                    if reviewer_corpus[0][i] in model.vocab:
                        sum_vector_text = sum_vector_text + model.wv[reviewer_corpus[0][i]]
                        counter = counter + 1
                    else:
                        print(reviewer_corpus[0][i])
                if counter > 0:
                    average_Reviewer_vector_text = sum_vector_text/ counter
                    all_usernames.append(df_reviewers.username.iloc[j+1])
                    all_domains.append(reviewer_interests)
                    all_cosine_sims.append(cosine_similarity(np.array([paper_vec]), np.array([average_Reviewer_vector_text]))[0,0])
    return np.array(all_usernames), np.array(all_domains), np.array(all_cosine_sims)


#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 3 TF Gensim ####################


def Get_Top_Words_tf(Paper_interest, df_reviewers=df_reviewers,num_suggestions=5, num_top20=20):
    POI_PDF = [extract_text(Paper_interest)]
    text = str(POI_PDF)
    words =  Get_Lemma_Words(POI_PDF)
    #print(len(words))
    fdist = FreqDist(words)
    X = np.array(fdist.most_common())
    top20_tf = X[:num_top20,0]

    #df_reviewers.username.iloc[+1]

    return top20_tf



def Split_columns(t):
  txt = " ".join("".join([" " if ch in string.punctuation else ch for ch in t]).split())
  sol1 = np.char.split(txt, ' ')
  txt_arr  = array_of_lists_to_array(sol1)
  uniarr = np.unique(txt_arr)
  return uniarr


def summatation_bot(all_usernames, all_domains, all_num_matched_words, all_matched_words):
  length = len(all_usernames)
  message = 'Hello. \nI have found ' + str(length) + ' possible reviewers for this paper.' +'\n\n'
  for i in range(length):
    ps = 'I believe ' + all_usernames[i] + ' will make a good reviewer for this paper because they have matched ' + str(int(all_num_matched_words[i])) +  ' words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ' + str(all_matched_words[i]) +'.\nFrom their topics domain: ' + str(all_domains[i].replace('\n', ', ')) +'.\n'
    message = message + ps + '\n'
  print(message)



def W2V_Gensim_Processing_tf(top_arr, model, print_outside_corpus=True):
    texts = ''
    for i in range(len(top_arr)):
        texts = texts + top_arr[i] + ' ' 
    #print(t)

    #download('punkt') #tokenizer, run once
    #download('stopwords') #stopwords dictionary, run once
    stop_words = stopwords.words('english')

    texts = [texts]
    corpus = [preprocess_gensim(text) for text in texts]

    sum_vector_text = np.zeros((300))
    counter = 0
    for i in range(len(corpus[0])):
        if corpus[0][i] in model.vocab:
            sum_vector_text = sum_vector_text + model.wv[corpus[0][i]]
            counter = counter + 1
        else:
            if print_outside_corpus == True:
                print(corpus[0][i])
    average_vector_text = sum_vector_text/ counter
    return average_vector_text

    
#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 3 TF-IDF Gensim ####################

def Get_Top_Words_tf_idf(Paper_interest, df=df, df_reviewers=df_reviewers, num_suggestions=5, num_top20=20):
    POI_PDF = [extract_text(Paper_interest)]
    text = str(POI_PDF)
    words = Get_Lemma_Words(POI_PDF)
    #print(len(words))
    fdist = FreqDist(words)
    X = np.array(fdist.most_common())
    tf_idf_arr_names, tf_idf_arr_floats = determine_wiki_td_idf(X, df=df)
    #print('determined wiki')
    num_arr = np.array(tf_idf_arr_floats)
    tf_idf_arr_names_arr = np.array(tf_idf_arr_names)
    top20_tf_idf = tf_idf_arr_names_arr[np.argsort(num_arr)[:num_top20]]
    return top20_tf_idf








#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 4 Gensim ####################

def TopReviewers_W2V_Gensim(number, all_usernames, all_domains, all_cosine_sims):
    message = 'Hello.\n I have found ' +str(number) + ' possible reviewers for this paper.'+ '\n\n'
    for J in range(number):
        index = np.argsort(all_cosine_sims)[-1-J]
        #print(index)
        ps = 'I believe '+ colored(str(all_usernames[index]), 'green') + ' will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of ' + colored(str(all_cosine_sims[index])[:6], 'blue') + '. This reviewers domain interests are ' + colored(str(all_domains[index].replace('\n', ',')), 'red')
        message = message + ps + '\n\n'
    print(message)    

#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 6 Gensim ####################

def MakingPaperVector_Idea6_Gensim(arr, model, printer=True):
    sum_vector_text = np.zeros((300))
    count = 0
    for J in range(len(arr)):
        texts = arr[J]
        texts = texts
        texts = texts.replace('-\\n','')
        texts = texts.replace('-\n','')
        texts = texts.replace('\\n',' ')
        texts = texts.replace('\n', ' ')
    
        texts = [texts]
        corpus = [preprocess_gensim(text) for text in texts]
    
        sum_line_vector_text = np.zeros((300))
        counter = 0
        for i in range(len(corpus[0])):
            if corpus[0][i] in model.vocab:
                sum_vector_text = sum_vector_text + model.wv[corpus[0][i]]
                counter = counter + 1
            else:
                if printer == True:
                    print(corpus[0][i])
        average_line_vector_text = sum_line_vector_text/ counter
        sum_vector_text = sum_vector_text + average_line_vector_text
        count = count + 1
    average_vector_text = sum_vector_text / count
    
    return average_vector_text


#########################################################################################################################################################

#########################################################################################################################################################

################# spaCy ####################

def GetReviewer_Vectors_spacy(model,df_reviewers=df_reviewers):
    reviewer_vectors = np.zeros(((df_reviewers.shape[0]-1),300))
    for i in range(df_reviewers.shape[0]-1):
        #if i%10 == 0:
        #    print(i)
        if pd.isna(df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i]) == False:
            review_text = df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i].lower()
            review_text = review_text.replace('-\\n','')
            review_text = review_text.replace('\\n',' ')
            review_text = review_text.replace('\n', ' ')
        
            review_arr = []
            for token in model(review_text):
                if token.is_alpha == True:
                    if token.is_stop == False:
                        review_arr.append(str(token.lemma_).lower())
            review_arr = np.array(review_arr)
        
            review_str = ''
            for j in np.unique(review_arr):
                review_str = review_str + j +' '
        
        #print(model(review_str).vector.shape)
        #print(reviewer_vectors.shape)
            reviewer_vectors[i] = model(review_str).vector
    
    return reviewer_vectors

def GetCosineSims_spacy(doc_vec, review_vec, df_reviewers=df_reviewers):
    all_usernames = []
    all_domains = []
    all_cosine_sims = []
    for j in range(len(review_vec)):
        if pd.isna(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing']) == False:
            all_cosine_sims.append(cosine_similarity(np.array([doc_vec]), np.array([review_vec[j]]))[0,0])
            all_domains.append(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing'].lower())
            all_usernames.append(df_reviewers.iloc[j+1].username)
    all_usernames= np.array(all_usernames)
    all_domains= np.array(all_domains)
    all_cosine_sims= np.array(all_cosine_sims)
    
    return all_usernames, all_domains, all_cosine_sims

#def TopReviewers_spacy(number=5, all_usernames=all_usernames, all_domains=all_domains, all_cosine_sims=all_cosine_sims):
def TopReviewers_spacy(number, all_usernames, all_domains, all_cosine_sims):
    message = 'Hello.\n I have found ' +str(number) + ' possible reviewers for this paper.'+ '\n\n'
    for J in range(number):
        index = np.argsort(all_cosine_sims)[-1-J]
        #print(index)
        ps = 'I believe '+ colored(str(all_usernames[index]), 'green') + ' will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of ' + colored(str(all_cosine_sims[index])[:6], 'blue') + '. This reviewers domain interests are ' + colored(str(all_domains[index].replace('\n', ',')), 'red')
        message = message + ps + '\n\n'
    print(message)        
    
    
def W2V_spaCy_Processing(texts, green_text=True, lemma=True, unique=True):
    texts = texts.replace('-\\n','')
    texts = texts.replace('-\n','')
    texts = texts.replace('\\n',' ')
    texts = texts.replace('\n', ' ')
    
    model = spacy.load('en_core_web_lg')
    reviewer_vectors = GetReviewer_Vectors_spacy(model)
    
    # Only Green Text: 
    if green_text == True:
        print('### Only Green Text - No Other Preprocessing ###')
        all_usernames, all_domains, all_cosine_sims = GetCosineSims_spacy(model(texts).vector, reviewer_vectors)
        TopReviewers_spacy(5, all_usernames, all_domains, all_cosine_sims)
        
    doc = model(texts)
    doc_arr = []
    for token in doc:
        if token.is_alpha == True:
            if token.is_stop == False:
                doc_arr.append(str(token.lemma_).lower())
    doc_arr = np.array(doc_arr)
    
    doc_arr_mod = ''
    for i in doc_arr:
        doc_arr_mod = doc_arr_mod + i +' '
    
    # Green Text and Lemmasation
    if lemma == True:
        print('### Green Text & Lemmasation ###')
        all_usernames2, all_domains2, all_cosine_sims2 = GetCosineSims_spacy(model(doc_arr_mod).vector, reviewer_vectors)
        TopReviewers_spacy(5, all_usernames2, all_domains2, all_cosine_sims2)
        
    if unique == True:
        print('### Green Text & Lemmasation & Unique Words ###')
        doc_str = ''
        for i in np.unique(doc_arr):
            doc_str = doc_str + i +' '
        all_usernames3, all_domains3, all_cosine_sims3 = GetCosineSims_spacy(model(doc_str).vector, reviewer_vectors)
        TopReviewers_spacy(5,all_usernames3, all_domains3, all_cosine_sims3)
        
def GetReviewer_Vectors(df_reviewers=df_reviewers):
    reviewer_vectors = np.zeros(((df_reviewers.shape[0]-1),300))
    for i in range(df_reviewers.shape[0]-1):
        #if i%10 == 0:
        #    print(i)
        if pd.isna(df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i]) == False:
            review_text = df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i].lower()
            review_text = review_text.replace('-\\n','')
            review_text = review_text.replace('\\n',' ')
            review_text = review_text.replace('\n', ' ')
        
            review_arr = []
            for token in model(review_text):
                if token.is_alpha == True:
                    if token.is_stop == False:
                        review_arr.append(str(token.lemma_).lower())
            review_arr = np.array(review_arr)
        
            review_str = ''
            for j in np.unique(review_arr):
                review_str = review_str + j +' '
        
        #print(model(review_str).vector.shape)
        #print(reviewer_vectors.shape)
            reviewer_vectors[i] = model(review_str).vector
    
    return reviewer_vectors

def GetCosineSims(doc_vec, review_vec, df_reviewers=df_reviewers):
    all_usernames = []
    all_domains = []
    all_cosine_sims = []
    for j in range(len(review_vec)):
        if pd.isna(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing']) == False:
            all_cosine_sims.append(cosine_similarity(np.array([doc_vec]), np.array([review_vec[j]]))[0,0])
            all_domains.append(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing'].lower())
            all_usernames.append(df_reviewers.iloc[j+1].username)
    all_usernames= np.array(all_usernames)
    all_domains= np.array(all_domains)
    all_cosine_sims= np.array(all_cosine_sims)
    
    return all_usernames, all_domains, all_cosine_sims

def Get_Paper_Vector_Idea6_SpaCy(arr,model):
    sum_vector_text = np.zeros((300))
    count = 0
    for J in range(len(arr)):
        sum_line_vector_text = model(str(arr[J])).vector
        counter = len(arr[J].split())
        
        average_line_vector_text = sum_line_vector_text/ counter
        sum_vector_text = sum_vector_text + average_line_vector_text
        count = count + 1
        
    average_vector_text = sum_vector_text / count
    return average_vector_text


#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 3 TF spaCy ####################
    


#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 3 TF-IDF spaCy ####################



#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 4 spaCy ####################

# No Functions

#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 6 spaCy ####################


    
def Idea6_lemma_Text(arr):
    text_list = []
    for J in range(len(arr)):
        doc = model(str(arr[J]))
        
        doc_arr = []
        for token in doc:
            if token.is_alpha == True:
                if token.is_stop == False:
                    doc_arr.append(str(token.lemma_).lower())
        doc_arr = np.array(doc_arr)
        
        doc_arr_mod = ''
        for i in doc_arr:
            doc_arr_mod = doc_arr_mod + i +' '
        #print(doc_arr_mod)
        text_list.append(doc_arr_mod)
    return text_list

def Idea6_unique_lemma_Text(arr):
    text_list = []
    for J in range(len(arr)):
        doc = model(str(arr[J]))
        
        doc_arr = []
        for token in doc:
            if token.is_alpha == True:
                if token.is_stop == False:
                    doc_arr.append(str(token.lemma_).lower())
        doc_arr = np.array(doc_arr)
        doc_str = ''
        for i in np.unique(doc_arr):
            doc_str = doc_str + i +' '
        text_list.append(doc_str)
    return text_list




#########################################################################################################################################################

#########################################################################################################################################################

################# Idea 9 Sense2Vec ####################


def GetPaperVector_Sense2Vec(text):
    doc = model(text)
    doc_arr = []
    pos_arr = []
    tag_arr = []
    dep_arr = []
    for token in doc:
        if token.is_alpha == True:
            if token.is_stop == False:
                doc_arr.append(str(token.lemma_).lower())
                pos_arr.append(str(token.pos_))
                tag_arr.append(str(token.tag_))
                dep_arr.append(str(token.dep_))
            
    doc_arr = np.array(doc_arr)
    pos_arr = np.array(pos_arr)
    tag_arr = np.array(tag_arr)
    dep_arr = np.array(dep_arr)

    word_vec = np.zeros((128))
    counter = 0
    for P in range(len(doc_arr)):
        best = s2v.get_best_sense(doc_arr[P])
        if best != None:
            vector = s2v[best]
            word_vec = word_vec + vector
            counter = counter + 1
    average_word_vec = word_vec / counter
    
    return average_word_vec



def GetReviewerSample_Sense2Vec(paper_vec, df_reviewers=df_reviewers):
    all_usernames = []
    all_domains = []
    all_cosine_sims = []
    for j in range(df_reviewers.shape[0]-1):
        if pd.isna(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing']) == False:
            reviewer_interests = df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing'].lower()
            reviewer_interests.replace('/',' ')
            doc = model(reviewer_interests)
            reviewer_arr = []
            for token in doc:
                if token.is_alpha == True:
                    if token.is_stop == False:
                        reviewer_arr.append(str(token.lemma_).lower())
            reviewer_arr = np.array(reviewer_arr)
            word_vec = np.zeros((128))
            counter = 0
            for P in range(len(reviewer_arr)):
                best = s2v.get_best_sense(reviewer_arr[P])
                if best != None:
                    vector = s2v[best]
                    word_vec = word_vec + vector
                    counter = counter + 1
            
            if counter > 0:
                average_reviewer_vec = word_vec / counter
          
                all_usernames.append(df_reviewers.username.iloc[j+1])
                all_domains.append(reviewer_interests)
                all_cosine_sims.append(cosine_similarity(np.array([paper_vec]), np.array([average_reviewer_vec]))[0,0])
    return np.array(all_usernames), np.array(all_domains), np.array(all_cosine_sims)
 
def TopReviewers(number, all_usernames, all_domains, all_cosine_sims):
    message = 'Hello.\nI have found ' +str(number) + ' possible reviewers for this paper.'+ '\n\n'
    for J in range(number):
        index = np.argsort(all_cosine_sims)[-1-J]
        #print(index)
        ps = 'I believe '+ colored(str(all_usernames[index]), 'green') + ' will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of ' + colored(str(all_cosine_sims[index])[:6], 'blue') + '. This reviewers domain interests are ' + colored(str(all_domains[index].replace('\n', ',')), 'red')
        message = message + ps + '\n\n'
    print(message) 

# Idea 2

In [133]:
all_usernames_Idea_2, all_domains_Idea_2, all_num_matched_words_Idea_2, all_matched_words_Idea_2 = Get_reviewer_sample_tf_idf(Paper_interest)

In [134]:
summatation_bot_tf_idf(all_usernames_Idea_2, all_domains_Idea_2, all_num_matched_words_Idea_2, all_matched_words_Idea_2)

Hello. 
I have found 5 possible reviewers for this paper.

I believe sbacchio will make a good reviewer for this paper because they have matched 2 words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ['computational', 'iterative'].
From their topics domain: computational physics,  high-performance computing,  lattice qcd,  quantum field theory,  iterative linear solvers.

I believe SaumikDana will make a good reviewer for this paper because they have matched 2 words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ['computational', 'python'].
From their topics domain: geology, geophysics, earth surface dynamics, hydrology, landslides, natural hazards, terrain analysis ,  computational science and engineering, geophysics, high-performance computing ,  computational fracture mechanics, applied mathematics, c++, asynchronous and task-based programming ,  mechanics,

# spaCy

In [150]:
model = spacy.load('en_core_web_lg')

# Idea 3 TF (spaCy)

In [151]:
top20_tf = Get_Top_Words_tf(Paper_interest)

In [152]:
top20_tf

array(['cardiac', 'problem', 'fenics', 'journal', 'et', 'package',
       'based', 'solving', 'boundary', 'python', 'pulse', 'code',
       'custom', 'material', 'using', 'computational', 'mechanic',
       'appropriate', 'user', 'need'], dtype='<U21')

In [153]:
reviewer_vectors = GetReviewer_Vectors()

In [154]:
doc_top20= ''
for i in top20_tf:
    doc_top20 = doc_top20 + i +' '

In [155]:
all_usernames_spacy_Idea_3_TF, all_domains_spacy_Idea_3_TF, all_cosine_sims_spacy_Idea_3_TF = GetCosineSims(model(doc_top20).vector, reviewer_vectors)

In [156]:
TopReviewers(5, all_usernames_spacy_Idea_3_TF, all_domains_spacy_Idea_3_TF, all_cosine_sims_spacy_Idea_3_TF)

Hello.
I have found 5 possible reviewers for this paper.

I believe michaelberks will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.8870. This reviewers domain interests are medical imaging, image processing, compute vision, machine learning/ai applied to images, general scientific/numerical data processing (unless requiring specialist domain specific knowledge outside of the areas listed above)

I believe aaronpeikert will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.8647. This reviewers domain interests are machine learning, - model selection, - structural equation modelling, - multimodel inference, - hierarchical data, , open science, - reproducibility, - dynamic document generation, - containers, - version control

I believe jarrah42 will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.8605. Th

# Idea 9 (Sense2Vec)

In [167]:
from sense2vec import Sense2Vec

In [168]:
s2v = Sense2Vec().from_disk("../s2v_old")

In [169]:
model = spacy.load('en_core_web_lg')

In [170]:
texts, arr = MakeGreenRedText(Paper_interest,False)

In [171]:
average_word_vec = GetPaperVector_Sense2Vec(texts)

In [172]:
all_usernames_sense2vec, all_domains_sense2vec, all_cosine_sims_sense2vec = GetReviewerSample_Sense2Vec(average_word_vec)

In [173]:
TopReviewers(5, all_usernames_sense2vec, all_domains_sense2vec, all_cosine_sims_sense2vec )

Hello.
I have found 5 possible reviewers for this paper.

I believe aaronpeikert will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.9277. This reviewers domain interests are machine learning, - model selection, - structural equation modelling, - multimodel inference, - hierarchical data, , open science, - reproducibility, - dynamic document generation, - containers, - version control

I believe torressa will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.9271. This reviewers domain interests are optimization: modelling (lps, milps, mmilps), solution approaches (heuristics, hyper-heuristics, column generation), applications (aviation, vehicle routing, scheduling)

I believe alexpghayes will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.9260. This reviewers domain interests are i'm interested in maki